In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pylab as plt
import sys, gc, warnings, random, math, time, datetime 
from tqdm import tqdm

from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error

import xgboost as xgb
import lightgbm as lgb

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

import os
print(os.listdir("../input/"))

['train_transaction.csv', 'train_identity.csv', 'ieee-fe-with-some-eda', 'separated', 'sample_submission.csv', 'ieee-data-minification', 'reduced', 'standalone-train-and-test-preprocessing', 'test_identity.csv', 'mini-model-predictions', 'mini-model-data', 'test_transaction.csv', 'mini-model-features', 'ieee-fe-for-local-test']


In [2]:
from utils import *

In [3]:
SEED = 42
seed_everything(SEED)

In [4]:
# loading data
train_df = pd.read_csv('../input/train_transaction.csv', index_col='TransactionID')
test_df = pd.read_csv('../input/test_transaction.csv', index_col='TransactionID')
test_df['isFraud'] = 0

train_identity = pd.read_csv('../input/train_identity.csv', index_col='TransactionID')
test_identity = pd.read_csv('../input/test_identity.csv', index_col='TransactionID')

In [5]:
train_df = reduce_mem_usage(train_df)
test_df  = reduce_mem_usage(test_df)

train_identity = reduce_mem_usage(train_identity)
test_identity  = reduce_mem_usage(test_identity)

Mem. usage decreased to 544.60 Mb (69.3% reduction)
Mem. usage decreased to 475.00 Mb (68.8% reduction)
Mem. usage decreased to 26.41 Mb (41.5% reduction)
Mem. usage decreased to 25.98 Mb (41.5% reduction)


In [6]:
print(train_df['ProductCD'].unique())

['W' 'H' 'C' 'S' 'R']


In [7]:
# W
train_df_w = train_df[train_df['ProductCD'] == 'W']
test_df_w = test_df[test_df['ProductCD'] == 'W']
print(train_df_w.shape)
print(test_df_w.shape)

print('W fraud rate is: ',train_df_w['isFraud'].sum() / len(train_df_w) * 100, '%')

train_df_w.to_pickle('../input/mini-model-data/train_df_w.pkl')
test_df_w.to_pickle('../input/mini-model-data/test_df_w.pkl')

(439670, 393)
(360987, 393)
W fraud rate is:  2.0399390451929857 %


In [8]:
# H
train_df_h = train_df[train_df['ProductCD'] == 'H']
test_df_h = test_df[test_df['ProductCD'] == 'H']
print(train_df_h.shape)
print(test_df_h.shape)

print('H fraud rate is: ',train_df_h['isFraud'].sum() / len(train_df_h) * 100, '%')

train_df_h.to_pickle('../input/mini-model-data/train_df_h.pkl')
test_df_h.to_pickle('../input/mini-model-data/test_df_h.pkl')

(33024, 393)
(29373, 393)
H fraud rate is:  4.766230620155039 %


In [5]:
# C
train_df_c = train_df[train_df['ProductCD'] == 'C']
test_df_c = test_df[test_df['ProductCD'] == 'C']
print(train_df_c.shape)
print(test_df_c.shape)

print('C fraud rate is: ',train_df_c['isFraud'].sum() / len(train_df_c) * 100, '%')

train_df_c.to_pickle('../input/mini-model-data/train_df_c.pkl')
test_df_c.to_pickle('../input/mini-model-data/test_df_c.pkl')

(68519, 393)
(69266, 393)
C fraud rate is:  11.687269224594637 %


In [7]:
train_df_c.head(20)

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987010,0,86549,75.887,C,16496,352.0,117.0,mastercard,134.0,credit,NaN,NaN,NaN,NaN,gmail.com,gmail.com,1.0,4.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,83.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,M0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,4.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,0.0,0.0,1.0,2.0,2.0,0.0,0.0,4.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,0.0,0.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,3.0,4.0,4.0,2.0,1.0,2.0,2.0,4.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,166.215393,166.215393,166.215393,90.327904,31.841299,90.327904,90.327904,90.327904,90.327904,0.0,0.0,0.0,75.887497,75.887497,75.887497,3.0,3.0,3.0,3.0,4.0,4.0,2.0,2.0,2.0,0.0,2.0,4.0,4.0,4.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,166.215393,166.215393,166.215393,90.327904,90.327904,90.327904,0.0,90.327904,90.327904,90.327904,0.0,0.0,0.0,75.887497,75.887497,75.887497,3.0,3.0,3.0,4.0,4.0,2.0,2.0,1.0,2.0,2.0,2.0,4.0,4.0,4.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,166.215393,166.215393,166.215393,90.327904,90.327904,31.841299,90.327904,90.327904,90.327904,90.327904,0.0,0.0,0.0,75.88

In [10]:
# S
train_df_s = train_df[train_df['ProductCD'] == 'S']
test_df_s = test_df[test_df['ProductCD'] == 'S']
print(train_df_s.shape)
print(test_df_s.shape)

print('S fraud rate is: ',train_df_s['isFraud'].sum() / len(train_df_s) * 100, '%')

train_df_s.to_pickle('../input/mini-model-data/train_df_s.pkl')
test_df_s.to_pickle('../input/mini-model-data/test_df_s.pkl')

(11628, 393)
(11418, 393)
S fraud rate is:  5.8995528035775715 %


In [11]:
# R
train_df_r = train_df[train_df['ProductCD'] == 'R']
test_df_r = test_df[test_df['ProductCD'] == 'R']
print(train_df_r.shape)
print(test_df_r.shape)

print('R fraud rate is: ',train_df_r['isFraud'].sum() / len(train_df_r) * 100, '%')

train_df_r.to_pickle('../input/mini-model-data/train_df_r.pkl')
test_df_r.to_pickle('../input/mini-model-data/test_df_r.pkl')

(37699, 393)
(35647, 393)
R fraud rate is:  3.782593702750736 %


In [12]:
train_identity.to_pickle('../input/mini-model-data/train_identity.pkl')
test_identity.to_pickle('../input/mini-model-data/test_identity.pkl')